# Modello convolutivo di base
In questo notebook costruiamo un **modello convolutivo di base** così da poter avere un'idea migliore del task che stiamo approcciando. Utilizzando un modello di base riusciamo a capire soprattutto che prestazioni riusciamo a ottenere senza troppi sforzi.

In [6]:
import tensorflow as tf

In [7]:
train_ds = tf.keras.utils.audio_dataset_from_directory(directory='../reduced_dataset/dataset/audio')

Found 12933 files belonging to 30 classes.
